In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Preparing data**

In [ ]:
train = pd.read_csv('../input/human-activity-recognition-with-smartphones/train.csv')
train.head()

In [ ]:
train.shape

In [ ]:
train.isnull().values.any()

In [ ]:
test = pd.read_csv('../input/human-activity-recognition-with-smartphones/test.csv')
test.head()

In [ ]:
print(test.shape)
test.isnull().values.any()

In [ ]:
X_train = train.iloc[:,:-2]
Y_train = train.iloc[:,-1]

print(X_train.shape)
print(Y_train.shape)

In [ ]:
X_test = test.iloc[:,:-2]
Y_test = test.iloc[:,-1]

print(X_test.shape)
print(Y_test.shape)

In [ ]:
Category_counts = np.array(Y_train.value_counts())
Category_counts

**There are five different activities i.e 'Standing','Sitting','Laying','Walking','Walking_downstairs','Walking_upstairs'.**

**Plotting a count plot of each activity in the training data.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,8))
sns.countplot(train.Activity)
plt.xticks(rotation=45)


**Creating a scatter plot using t-SNE**

Using t-SNE data can be visualized from a extremely high dimensional space to a low dimensional space and still it retains lots of actual information. Given training data has 562 unqiue features, using t-SNE let's visualize it to a 2D space.

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state = 42, n_components=2, verbose=1, perplexity=50, n_iter=1000).fit_transform(X_train)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x =tsne[:, 0], y = tsne[:, 1],data = train,hue = train["Activity"])


In [ ]:
train['tBodyAcc-mean()-X'].hist()

In [ ]:
train['tBodyAcc-mean()-Y'].hist()

In [ ]:
train['tBodyAcc-mean()-Z'].hist()

In [ ]:
#Y_train = Y_train.reshape((-1,1))
#Y_test = Y_test.reshape((-1,1))

#print(Y_train.shape)
#print(Y_test.shape)

**Scaling the data**

   **Creating labels for different classes**

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)

le.classes_

**It is necessary to create a one-hot vector for classes to fit the data in the model.**

In [ ]:
Y_train = pd.get_dummies(Y_train).values
Y_test = pd.get_dummies(Y_test).values

In [ ]:
Y_train

In [ ]:
Y_train.shape



**Creating our model**

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense,Dropout

model = models.Sequential()

model.add(Dense(64,activation='relu',input_dim=X_train.shape[1]))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='relu'))
model.add(Dense(6,activation='softmax'))

model.summary()


**Compiling and training the model.**

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train,Y_train,epochs=30,batch_size = 128,validation_split=0.3)

**Visualising loss and accuracy curve of the model.**

In [ ]:
plt.plot(hist.history['loss'],label='train_loss')
plt.plot(hist.history['val_loss'],label='val_loss')
plt.xlabel('Epochs',fontsize=18)
plt.ylabel('Loss',fontsize=18)
plt.legend()
plt.title('Loss Curve',fontsize=22)
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'],label='train_accuracy')
plt.plot(hist.history['val_accuracy'],label='val_accuracy')
plt.xlabel('Epochs',fontsize=18)
plt.ylabel('Accuracy',fontsize=18)
plt.legend()
plt.title('Accuracy Curve',fontsize=22)
plt.show()

In [ ]:
model.save('my_model.h5')

**Making predictions on test data**

In [ ]:
predict = model.predict(X_test)

predictions = np.argmax(predict,axis=1)



In [ ]:
predictions

In [ ]:
Y_test = np.argmax(Y_test,axis=1)

**Calculating accuracy**

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

conf_matrix = confusion_matrix(Y_test,predictions)
plot_confusion_matrix(conf_matrix)

precision = precision_score(Y_test,predictions,average='weighted')
recall = recall_score(Y_test, predictions,average='weighted')
accuracy = accuracy_score(Y_test,predictions)

print("Accuracy = "+str(accuracy))
print("Precision = "+str(precision))
print("Recall = "+str(recall))

**The model was able to produce 93% accurate results.**